# Calculate CESM Indeces

This file grabs the monthly CESM indeces, and calculates the averages for the time periods of interest (aka the warm season), as set in the _indexDefitionions file

You need to run 'calculateCesmMonthlyIndeces.ipnyb' first

Theres a few examples of output at the bottom to sanity check this is meaningful

In [1]:
#import my functions
import utils._modelDefinitions as _model
import utils._indexDefinitions as _index
from utils.timePeriod import averageForTimePeriod

In [2]:
import xarray
import matplotlib.pyplot as plt

The indeces for each month have already been calculated. So loading up the saves versions, and putting them into a dict of pandas arrays.

In [ ]:
timePIndeces=dict()

for experiment in _model.cesmAll:
    print(experiment)
    indeces = xarray.load_dataset('results/cesmMonthlyIndeces/' + experiment + '.nc')

    timePIndeces[experiment] = averageForTimePeriod(indeces)



0850cntl.001
LME.002
LME.003
LME.008
LME.009
001
002
003
004
005
006
007
008
009
010
011
012


To make this all easier to deal with, convert the result into an xarray and add some extra relevant information as attributes.

This allows for filtering by index, year or experiment 

In [ ]:
# so i've just written the results into a dict of pandas array. This is kinda funky because I can't slice one index across multiple models
# instead, put the whole thing into an xarray with a dimension for the experiment
results=xarray.concat(
    [
        xarray.DataArray(timePIndeces[experiment], 
                         dims=['year','index'], 
                         coords={
                             'year':timePIndeces[experiment].index,
                             'index':timePIndeces[experiment].columns,
                             'experiment': experiment})
    for experiment in _model.cesmAll 
    ],
    dim='experiment'
)

results.assign_attrs(indeces.attrs)
results.assign_attrs(_index.monthsOfInterest)



Save the result for use later

In [ ]:
results.to_netcdf('results/cesmTradIndeces.nc')

Some sanity checks to see if it looks ok:


In [ ]:
# plotSomething
fig = plt.figure(figsize=(12,8))
ax = fig.add_subplot(1,1,1)

results.sel(experiment='001', index='nino34').plot(label='001')
results.sel(experiment='LME.002', index='nino34').plot(label='LME.002')

#Tidy up a bit
plt.title("CESM Full Forcing Nino34 Ensemble")
plt.ylim(-4,4)
plt.minorticks_on()
plt.legend()

In [ ]:
# A multi-experiment mean?

mean = results.sel(index='nino34').mean(dim='experiment')
std = results.sel(index='nino34').std(dim='experiment')

In [ ]:
plt.figure()
mean.plot()
std.plot()

In [ ]:
# why limit yourself to one index?

mean = results.sel(experiment=_model.cesmFullForcings).mean(dim='experiment')
std = results.sel(experiment=_model.cesmFullForcings).std(dim='experiment')

In [ ]:
std